In [10]:
# pip install torchvision

In [9]:
# pip install visdom

In [4]:
# pip install --upgrade pip

In [8]:
!python3.6 -m visdom.server

/usr/local/bin/python3.6: Error while finding module specification for 'visdom.server' (ModuleNotFoundError: No module named 'visdom')


In [30]:
from platform import python_version

print(python_version())

3.9.10


In [33]:
!python3 -m visdom.server

/usr/local/bin/python3.6: Error while finding module specification for 'visdom.server' (ModuleNotFoundError: No module named 'visdom')


In [1]:
import torch
import os, glob
import random, csv

import visdom
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from PIL import Image

import visdom
import time

In [2]:
class Face(Dataset):

    def __init__(self, root, resize, mode):
        super(Face, self).__init__()

        self.root = root
        self.resize = resize

        self.name2label = {}
        for name in sorted(os.listdir(os.path.join(root))):
            if not os.path.isdir(os.path.join(root, name)):
                continue

            self.name2label[name] = len(self.name2label.keys())

        print(self.name2label)

        # image, label
        self.images, self.labels = self.load_csv('images.csv')

        if mode == 'train':  # 60%
            self.images = self.images[:int(0.6 * len(self.images))]
            self.labels = self.labels[:int(0.6 * len(self.labels))]

        elif mode == 'val':  # 20% (60-80%)
            self.images = self.images[int(0.6 * len(self.images)):int(0.8 * len(self.images))]
            self.labels = self.labels[int(0.6 * len(self.labels)):int(0.8 * len(self.labels))]

        elif mode == 'test':  # 20% (80-100%)
            self.images = self.images[int(0.8 * len(self.images)):]
            self.labels = self.labels[int(0.8 * len(self.labels)):]

    def load_csv(self, filename):

        if not os.path.exists(os.path.join(self.root, filename)):
            images = []
            for name in self.name2label.keys():
                # 'pokemon\\bulbasaur\\00000000.png'
                images += glob.glob(os.path.join(self.root, name, '*.png'))
                images += glob.glob(os.path.join(self.root, name, '*.jpg'))
                images += glob.glob(os.path.join(self.root, name, '*.jpeg'))

            # 1167, 'pokemon\\bulbasaur\\00000000.png'
            print(len(images), images)

            random.shuffle(images)
            with open(os.path.join(self.root, filename), mode='w', newline='') as f:
                writer = csv.writer(f)
                for img in images:
                    name = img.split(os.sep)[-2]
                    label = self.name2label[name]
                    # 'pokemon\\bulbasaur\\00000000.png', 0
                    writer.writerow([img, label])
                print(f"written into csv file: {filename}")

        # read from csv file
        images, labels = [], []
        with open(os.path.join(self.root, filename)) as f:
            reader = csv.reader(f)
            for row in reader:
                # 'pokemon\\bulbasaur\\00000000.png', 0
                img, label = row
                label = int(label)

                images.append(img)
                labels.append(label)

        assert len(images) == len(labels)

        return images, labels

    def __len__(self):
        return len(self.images)

    def denormalize(self, x_hat):
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        # x_hat = (x-mean)/std
        # x = x_hat * std + mean
        # x: [c, h, w]
        # mean: [3] => [3, 1, 1] for broadcast to happen
        mean = torch.tensor(mean).unsqueeze(1).unsqueeze(1)
        std = torch.tensor(std).unsqueeze(1).unsqueeze(1)

        x = x_hat * std + mean

        return x

    def __getitem__(self, idx):
        # idx: [0:len(images)]
        # self.images, self.labels
        # img: 'pokemon\\bulbasaur\\00000000.png'
        # label: 0
        img, label = self.images[idx], self.labels[idx]

        tf = transforms.Compose([
            lambda x: Image.open(x).convert('RGB'),  # str path -> img data
            transforms.Resize((int(self.resize * 1.25), int(self.resize * 1.25))),
            transforms.RandomRotation(15),
            transforms.CenterCrop(self.resize),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])  # from imagenet
        ])

        img = tf(img)
        label = torch.tensor(label)

        return img, label

In [3]:
viz = visdom.Visdom()
# using self implemented Dataset class
# db = Face('data', 224, 'train')    
# print(db.images[0], db.labels[0])

# x, y = next(iter(db))
# print(x.shape, y)

# viz.image(db.denormalize(x), win='sample_x', opts=dict(title='sample_x'))

# loader = DataLoader(db, batch_size=32, shuffle=True)

# for x, y in loader:
    # viz.images(db.denormalize(x), nrow=8, win='batch', opts=dict(title='batch'))
    # viz.text(str(y.numpy()), win='label', opts=dict(title='batch_y'))

    # time.sleep(10)

Setting up a new session...
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/usr/local/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/usr/local/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 61] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/usr/local/lib/python3.9/site-packages/urllib3/connectionpool.py", line 398, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/local/lib/python3.9/site-packages/urllib3/connection.py", line 239, in request
    super(H

Exception in user code:
------------------------------------------------------------


In [14]:
print("done")

done


In [21]:
from PIL import Image
import numpy as np
import os

def load_image(infilename):
    """This function loads an image into memory when you give it
       the path of the image
    """
    img = Image.open(infilename)
    img.load()
    data = np.asarray(img, dtype="float32")
    return data, data.shape


def create_npy_from_image(images_folder, output_name, num_images, image_dim):
    """Loops through the images in a folder and saves all of them
       as a numpy array in output_name
    """
    image_matrix = np.empty((num_images, image_dim, image_dim, 3), dtype=np.float32)
    for i, filename in enumerate(os.listdir(images_folder)):
        if filename.endswith(".jpg"):
            data = load_image(images_folder + filename)
            image_matrix[i] = data
        elif filename.endswith(".jpeg"):
            data = load_image(images_folder + filename)
            image_matrix[i] = data
        else:
            continue
    np.save(output_name, image_matrix)

In [23]:
img, shape = load_image('./data/cloth-mask/0018.jpg')

In [11]:
create_npy_from_image("./data/cloth-mask", output_name, num_images, shape)

Code           README.md      cnn_mask.ipynb data
